In [ ]:
import os
from ipywidgets import Output, GridspecLayout
from IPython import display

def parse_experiment_parameters(experiment_folder):
    start_frame, fps, duration = map(int, experiment_folder.split('_')[-4:-1])
    return {"start_frame": start_frame, "fps": fps, "duration": duration}

def select_experiment(experiment_folders, fields_and_values):
    selection = []
    for experiment_folder in experiment_folders:
        params = parse_experiment_parameters(experiment_folder)
        is_fine = True
        for field, normal_values_list in fields_and_values.items():
            if params[field] not in normal_values_list:
                is_fine = False
                break
        if is_fine:
            selection.append(experiment_folder)
    return selection
              

def present_experiments(experiments, embed=False, omit_source=False):
    grid = GridspecLayout(len(experiments), 2 if omit_source else 3)

    for i, experiment_folder in enumerate(experiments):
        if not omit_source:
            input_video = f'./{experiment_folder}/rgb_converted.mp4'
            if not os.path.exists(input_video):
                os.system(f"ffmpeg -i ./{experiment_folder}/rgb.mp4 -vcodec libx264 -acodec aac ./{experiment_folder}/rgb_converted.mp4")
            assert(os.path.exists(input_video))
            
        output_video = f'./{experiment_folder}/output/output.mp4'
        assert(os.path.exists(output_video))
        
        out = Output()
        with out:
            props = parse_experiment_parameters(experiment_folder)
            prop_str = ", ". join(f"*{k}* = {v}" for k, v in props.items())
            with open(f'./{experiment_folder}/output/output.txt') as f:
                prompt = f.read()
            display.display(display.Markdown(prop_str + f"\n\n*prompt*={prompt}"))
        grid[i, 0] = out
        
        if not omit_source:
            out = Output()
            with out:
                display.display(display.Video(input_video, embed=embed))
            grid[i, 1] = out
        out = Output()
        with out:
            display.display(display.Video(output_video, embed=embed))
        grid[i, 1 if omit_source else 2] = out
    return grid
        
experiment_folders = ['2024_08_15__19_55_49_0_4_2_full',
 '2024_08_15__19_55_49_6000_4_2_full',
 '2024_08_15__19_55_49_0_4_4_full',
 '2024_08_15__19_55_49_6000_4_4_full',
 '2024_08_15__19_55_49_0_4_8_full',
 '2024_08_15__19_55_49_6000_4_8_full',
 '2024_08_15__19_55_49_0_30_2_full',
 '2024_08_15__19_55_49_6000_30_2_full',
 '2024_08_15__19_55_49_0_30_4_full',
 '2024_08_15__19_55_49_6000_30_4_full',
 '2024_08_15__19_55_49_0_30_8_full',
 '2024_08_15__19_55_49_6000_30_8_full',
 '2024_08_15__19_55_49_12000_4_2_full',
 '2024_08_15__19_55_49_12000_4_4_full',
 '2024_08_15__19_55_49_12000_4_8_full',
 '2024_08_15__19_55_49_12000_30_2_full',
 '2024_08_15__19_55_49_12000_30_4_full',
 '2024_08_15__19_55_49_12000_30_8_full',
 '2024_08_15__19_55_49_18000_4_2_full',
 '2024_08_15__19_55_49_18000_4_4_full',
 '2024_08_15__19_55_49_18000_4_8_full',
 '2024_08_15__19_55_49_18000_30_2_full',
 '2024_08_15__19_55_49_18000_30_4_full',
 '2024_08_15__19_55_49_18000_30_8_full']

In [27]:
def export_experiments(experiments, out_html, pad=False, title=None):
    with open(out_html, 'a' if pad else 'w') as f:
        print('''<table><thead>  <meta charset="UTF-8"><style>
            table {
            width: 100vw;
            height: 100vh;
            border-collapse: collapse;
            table-layout: fixed; /* Ensures equal-width columns */
            }

            td {
            border: 1px solid #ccc;
            width: 50%; /* Adjust as needed */
            height: auto;
            padding: 10px;
            vertical-align: top;
            }

            .video-container {
            position: relative;
            width: 100%;
            padding-top: 56.25%; /* 16:9 aspect ratio */
            }

            .video-container iframe,
            .video-container video {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            }</style>''', file=f)
        if title is not None:
            print(f'<h1 style="text-align:center;">{title}</h1>', file=f)
        print('<tr><th>Info</th><th>Input</th><th>Output</th></tr></thead>', file=f)
        print('<tbody>', file=f)
        for experiment_folder in experiments:
            print('<tr>', file=f)
            input_video = f'./{experiment_folder}/rgb_converted.mp4'
            if not os.path.exists(input_video):
                os.system(f"ffmpeg -i ./{experiment_folder}/rgb.mp4 -vcodec libx264 -acodec aac ./{experiment_folder}/rgb_converted.mp4")
            output_video = f'./{experiment_folder}/output/output.mp4'
            assert(os.path.exists(input_video))
            assert(os.path.exists(output_video))
            
            props = parse_experiment_parameters(experiment_folder)
            prop_str = ", ". join(f"<i>{k}</i> = {v}" for k, v in props.items())
            with open(f'./{experiment_folder}/output/output.txt') as _f:
                prompt = _f.read()
                
            param_str = prop_str + f"</p><p><i>prompt</i>={prompt}" # markdown
            
            # table line with 3 items
            item_str = f'<td><p>{param_str}</p></td>'
            print(item_str, file=f)
            item_str = f'<td><div class="video-container"><video controls autoplay><source src={input_video} type="video/mp4"></video></div></td>'
            print(item_str, file=f)
            item_str = f'<td><div class="video-container"><video controls autoplay><source src={output_video} type="video/mp4"></video></div></td>'
            print(item_str, file=f)
            print('</tr>', file=f)
        print('</tbody>', file=f)

is_first = True
for fps in [30, 4]:
    for duration in [2, 4, 8]:
        export_experiments(select_experiment(experiment_folders, {"fps": [fps], "duration": [duration]}), 'index.html',
                           title=f'<i>fps</i>={fps}, <i>duration</i>={duration}',
                           pad=not is_first)
        is_first = False


In [9]:
# results on 30 fps, short-length videos (2s)
#fine 
present_experiments(select_experiment(experiment_folders, {"fps": [30], "duration": [2]}), embed=True)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [10]:
# results on 30 fps, medium-length videos (4s)
# fine
present_experiments(select_experiment(experiment_folders, {"fps": [30], "duration": [4]}), embed=True)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [11]:
# results on 30 fps, large-length videos (8s)
# hallucinations happens
present_experiments(select_experiment(experiment_folders, {"fps": [30], "duration": [8]}), embed=True)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [12]:
# results on 4 fps, short-length videos (2s)
present_experiments(select_experiment(experiment_folders, {"fps": [4], "duration": [2]}), embed=True)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [13]:
# results on 4 fps, medium-length videos (4s)
present_experiments(select_experiment(experiment_folders, {"fps": [4], "duration": [4]}), embed=True)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [14]:
# results on 4 fps, large-length videos (8s)
present_experiments(select_experiment(experiment_folders, {"fps": [4], "duration": [8]}), embed=True)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [38]:
present_experiments(['../2024_08_15__19_55_49_18000_30_4_full',
                     '../2024_08_15__19_55_49_18000_30_4_full1',
                     '../2024_08_15__19_55_49_18000_30_4_full2',
                     '../2024_08_15__19_55_49_18000_30_4_full3'], embed=True)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [37]:
present_experiments(['../2024_08_15__19_55_49_18000_30_4_full',
                     '../2024_08_15__19_55_49_18000_30_4_dptonly',
                     '../2024_08_15__19_55_49_18000_30_4_dptseg'], embed=True)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…